In [3]:
!pip install PyPDF2
!pip install reportlab

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import PyPDF2
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import tempfile

# Load model and tokenizer
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_response(prompt, max_length=1024):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response

def extract_text_from_pdf(pdf_file):
    if pdf_file is None:
        return ""
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def save_text_as_pdf(text, title="output"):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    c = canvas.Canvas(temp_file.name, pagesize=letter)
    width, height = letter
    y = height - 50
    for line in text.split("\n"):
        c.drawString(50, y, line)
        y -= 15
        if y < 50:  # new page if needed
            c.showPage()
            y = height - 50
    c.save()
    return temp_file.name

# === Feature 1: Eco Tips Generator ===
def eco_tips_generator(problem_keywords):
    prompt = f"Generate practical and actionable eco-friendly tips for sustainable living related to: {problem_keywords}. Provide specific solutions and suggestions."
    return generate_response(prompt, max_length=1000)

def eco_tips_with_pdf(keywords):
    tips = eco_tips_generator(keywords)
    pdf_path = save_text_as_pdf(tips, "eco_tips")
    return tips, pdf_path

# === Feature 2: Policy Summarization ===
def policy_summarization(pdf_file, policy_text):
    if pdf_file is not None:
        content = extract_text_from_pdf(pdf_file)
        summary_prompt = f"Summarize the following policy document and extract the most important points, key provisions, and implications:\n\n{content}"
    else:
        summary_prompt = f"Summarize the following policy document and extract the most important points, key provisions, and implications:\n\n{policy_text}"
    return generate_response(summary_prompt, max_length=1200)

def policy_summary_with_pdf(pdf_file, text_input):
    summary = policy_summarization(pdf_file, text_input)
    pdf_path = save_text_as_pdf(summary, "policy_summary")
    return summary, pdf_path

# === Feature 3: Carbon Footprint Estimator ===
def carbon_footprint_estimator(car_km, electricity_kwh, flights, diet_type):
    # Simple estimation (not exact science, just for demo)
    footprint = 0
    footprint += car_km * 0.21  # avg 0.21 kg CO2 per km
    footprint += electricity_kwh * 0.5  # avg 0.5 kg CO2 per kWh
    footprint += flights * 250  # avg short flight ~250kg CO2
    if diet_type == "Meat-heavy":
        footprint += 2000
    elif diet_type == "Mixed":
        footprint += 1500
    elif diet_type == "Vegetarian":
        footprint += 1000
    elif diet_type == "Vegan":
        footprint += 800

    advice = eco_tips_generator("reduce carbon footprint, " + diet_type)
    return f"🌍 Estimated Annual Carbon Footprint: {footprint:.2f} kg CO₂\n\nEco Advice:\n{advice}"

# === Feature 4: Policy Comparison ===
def policy_comparison(pdf1, text1, pdf2, text2):
    content1 = extract_text_from_pdf(pdf1) if pdf1 else text1
    content2 = extract_text_from_pdf(pdf2) if pdf2 else text2
    prompt = f"Compare the following two policy documents. Highlight similarities, differences, and potential conflicts.\n\nPolicy 1:\n{content1}\n\nPolicy 2:\n{content2}"
    return generate_response(prompt, max_length=1200)

# === Gradio UI ===
with gr.Blocks() as app:
    gr.Markdown("# 🌍 Eco Assistant & Policy Analyzer")

    with gr.Tabs():
        # Eco Tips Generator
        with gr.TabItem("Eco Tips Generator"):
            with gr.Row():
                with gr.Column():
                    keywords_input = gr.Textbox(
                        label="Environmental Problem/Keywords",
                        placeholder="e.g., plastic, solar, water waste, energy saving...",
                        lines=3
                    )
                    generate_tips_btn = gr.Button("Generate Eco Tips")
                with gr.Column():
                    tips_output = gr.Textbox(label="Sustainable Living Tips", lines=15)
                    tips_pdf = gr.File(label="Download Tips as PDF")
            generate_tips_btn.click(
                eco_tips_with_pdf, inputs=keywords_input, outputs=[tips_output, tips_pdf]
            )

        # Policy Summarization
        with gr.TabItem("Policy Summarization"):
            with gr.Row():
                with gr.Column():
                    pdf_upload = gr.File(label="Upload Policy PDF", file_types=[".pdf"])
                    policy_text_input = gr.Textbox(
                        label="Or paste policy text here",
                        placeholder="Paste policy document text...",
                        lines=5
                    )
                    summarize_btn = gr.Button("Summarize Policy")
                with gr.Column():
                    summary_output = gr.Textbox(label="Policy Summary & Key Points", lines=20)
                    summary_pdf = gr.File(label="Download Summary as PDF")
            summarize_btn.click(
                policy_summary_with_pdf,
                inputs=[pdf_upload, policy_text_input],
                outputs=[summary_output, summary_pdf]
            )

        # Carbon Footprint Estimator
        with gr.TabItem("Carbon Footprint Estimator"):
            with gr.Row():
                with gr.Column():
                    car_km = gr.Number(label="Car travel per year (km)", value=5000)
                    electricity_kwh = gr.Number(label="Electricity use per year (kWh)", value=3000)
                    flights = gr.Number(label="Flights per year", value=2)
                    diet_type = gr.Dropdown(
                        ["Meat-heavy", "Mixed", "Vegetarian", "Vegan"],
                        label="Diet type",
                        value="Mixed"
                    )
                    footprint_btn = gr.Button("Estimate Footprint")
                with gr.Column():
                    footprint_output = gr.Textbox(label="Carbon Footprint & Advice", lines=15)
            footprint_btn.click(
                carbon_footprint_estimator,
                inputs=[car_km, electricity_kwh, flights, diet_type],
                outputs=footprint_output
            )

        # Policy Comparison
        with gr.TabItem("Policy Comparison"):
            with gr.Row():
                with gr.Column():
                    pdf1 = gr.File(label="Upload Policy PDF 1", file_types=[".pdf"])
                    text1 = gr.Textbox(label="Or paste Policy Text 1", lines=5)
                    pdf2 = gr.File(label="Upload Policy PDF 2", file_types=[".pdf"])
                    text2 = gr.Textbox(label="Or paste Policy Text 2", lines=5)
                    compare_btn = gr.Button("Compare Policies")
                with gr.Column():
                    compare_output = gr.Textbox(label="Comparison Result", lines=20)
            compare_btn.click(
                policy_comparison,
                inputs=[pdf1, text1, pdf2, text2],
                outputs=compare_output
            )

app.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6e895d80ffadd75a0e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
